# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

In [1]:
import pandas as pd

with open ('female.txt') as f:
    female = f.read()
with open ('male.txt') as f:
    male = f.read()

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [2]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
for line in open ('male.txt'):
    p = morph.parse(line)[0]
    print(p)

Parse(word='aamir\n', tag=OpencorporaTag('LATN'), normal_form='aamir\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Aamir\n'),))
Parse(word='aaron\n', tag=OpencorporaTag('LATN'), normal_form='aaron\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Aaron\n'),))
Parse(word='abbey\n', tag=OpencorporaTag('LATN'), normal_form='abbey\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Abbey\n'),))
Parse(word='abbie\n', tag=OpencorporaTag('LATN'), normal_form='abbie\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Abbie\n'),))
Parse(word='abbot\n', tag=OpencorporaTag('LATN'), normal_form='abbot\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Abbot\n'),))
Parse(word='abbott\n', tag=OpencorporaTag('LATN'), normal_form='abbott\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Abbott\n'),))
Parse(word='abby\n', tag=OpencorporaTag('LATN'), normal_form='abby\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Abby\n'),))
Parse(word='abdel\n', tag=OpencorporaTag('LATN'), normal_form='abdel\n', score=1.0,

Parse(word='augustin\n', tag=OpencorporaTag('LATN'), normal_form='augustin\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Augustin\n'),))
Parse(word='augustine\n', tag=OpencorporaTag('LATN'), normal_form='augustine\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Augustine\n'),))
Parse(word='augusto\n', tag=OpencorporaTag('LATN'), normal_form='augusto\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Augusto\n'),))
Parse(word='augustus\n', tag=OpencorporaTag('LATN'), normal_form='augustus\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Augustus\n'),))
Parse(word='austen\n', tag=OpencorporaTag('LATN'), normal_form='austen\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Austen\n'),))
Parse(word='austin\n', tag=OpencorporaTag('LATN'), normal_form='austin\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Austin\n'),))
Parse(word='ave\n', tag=OpencorporaTag('LATN'), normal_form='ave\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Ave\n'),))
Parse(word='averell\n', tag=OpencorporaTag('LAT

Parse(word='martie\n', tag=OpencorporaTag('LATN'), normal_form='martie\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Martie\n'),))
Parse(word='martin\n', tag=OpencorporaTag('LATN'), normal_form='martin\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Martin\n'),))
Parse(word='martino\n', tag=OpencorporaTag('LATN'), normal_form='martino\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Martino\n'),))
Parse(word='marty\n', tag=OpencorporaTag('LATN'), normal_form='marty\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Marty\n'),))
Parse(word='martyn\n', tag=OpencorporaTag('LATN'), normal_form='martyn\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Martyn\n'),))
Parse(word='marv\n', tag=OpencorporaTag('LATN'), normal_form='marv\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Marv\n'),))
Parse(word='marve\n', tag=OpencorporaTag('LATN'), normal_form='marve\n', score=1.0, methods_stack=((<LatinAnalyzer>, 'Marve\n'),))
Parse(word='marven\n', tag=OpencorporaTag('LATN'), normal_form='marven\

С помощью pymorphy2 не удалось посмотреть к какому полу относится имя, pymorphy тоже определят пол очень плохо. Попробуем перевести имена на русский язык и снова применить pymorphy2. Используем для этого ф-ю translate, которая переведет имена из латиницы в кириллицу

In [5]:
def translate(name):
 
    #Заменяем пробелы и преобразуем строку к нижнему регистру
    name = name.replace(' ','-').lower()
 
    #
    transtable = (
        (u"Sch", u"Щ"),
        (u"SCH", u"Щ"),
        ( u"Yo", u"Ё"),
        ( u"YO", u"Ё"),
        ( u"Zh", u"Ж"),
        ( u"ZH", u"Ж"),
        ( u"Ts", u"Ц"),
        ( u"TS", u"Ц"),
        ( u"Ch", u"Ч"),
        ( u"CH", u"Ч"),
        ( u"Sh", u"Ш"),
        ( u"SH", u"Ш"),
        ( u"Yi", u"Ы"),
        ( u"YI", u"Ы"),
        ( u"Yu", u"Ю"),
        ( u"YU", u"Ю"),
        ( u"Ya", u"Я"),
        ( u"YA", u"Я"),
        ( u"A", u"А"),
        ( u"B", u"Б"),
        ( u"V", u"В"),
        ( u"G", u"Г"),
        ( u"D", u"Д"),
        ( u"E", u"Е"),
        ( u"Z", u"З"),
        ( u"I", u"И"),
        ( u"J", u"Й"),
        ( u"K", u"К"),
        ( u"L", u"Л"),
        ( u"M", u"М"),
        ( u"N", u"Н"),
        ( u"O", u"О"),
        ( u"P", u"П"),
        ( u"R", u"Р"),
        ( u"S", u"С"),
        ( u"T", u"Т"),
        ( u"U", u"У"),
        ( u"F", u"Ф"),
        ( u"H", u"Х"),
        ( u"E", u"Э"),
        ( u"sch", u"щ"),
        ( u"yo", u"ё"),
        ( u"zh", u"ж"),
        ( u"ts", u"ц"),
        ( u"ch", u"ч"),
        ( u"sh", u"ш"),
        ( u"yi", u"ы"),
        ( u"yu", u"ю"),
        ( u"ya", u"я"),
        ( u"a", u"а"),
        ( u"b", u"б"),
        ( u"v", u"в"),
        ( u"g", u"г"),
        ( u"d", u"д"),
        ( u"e", u"е"),
        ( u"z", u"з"),
        ( u"i", u"и"),
        ( u"j", u"й"),
        ( u"k", u"к"),
        ( u"l", u"л"),
        ( u"m", u"м"),
        ( u"n", u"н"),
        ( u"o", u"о"),
        ( u"p", u"п"),
        ( u"r", u"р"),
        ( u"s", u"с"),
        ( u"t", u"т"),
        ( u"u", u"у"),
        ( u"f", u"ф"),
        ( u"h", u"х"),
        ( u"e", u"э"),
    )
    #перебираем символы в таблице и заменяем
    for symb_in, symb_out in transtable:
        name = name.replace(symb_in, symb_out)
    #возвращаем переменную
    return name

In [6]:
femn = []
for line in female.split():
    p = morph.parse(translate(line))[0]
    g = p.tag.gender
    if p.tag.gender == 'femn':
        femn.append(line)
        #print(femn)

In [10]:
print(len(female.split()))
print(len(femn))

5004
2121


In [11]:
masc = []
for line in male.split():
    p = morph.parse(translate(line))[0]
    g = p.tag.gender
    if p.tag.gender == 'masc':
        masc.append(line)
        #print(masc)

In [12]:
print(len(male.split()))
print(len(masc))

2943
1765


In [13]:
c = ['Names']
f = pd.DataFrame(femn, columns=c)
m = pd.DataFrame(masc, columns=c)

In [14]:
f['Sex'] = "female"
m['Sex'] = "male"

In [15]:
f_train = f.iloc[:1580, :]
f_test = f.iloc[1580:, :]
m_train = m.iloc[:1580, :]
m_test = m.iloc[1580:, :]

In [16]:
df_train = pd.concat([f_train, m_train])
df_test = pd.concat([f_test, m_test])

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [17]:
from nltk.util import ngrams
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline

#### Mетод наивного Байеса

In [18]:
Val1 = 2
Val2 = 3

clf = Pipeline([
    ('vect',CountVectorizer(analyzer='char', ngram_range=(Val1,Val2))),
    ('clf',  MultinomialNB()),
])

In [19]:
clf.fit(df_train.Names, df_train.Sex)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [20]:
predicted = clf.predict(df_test.Names)

In [21]:
print("Accuracy: {0:6.2f}".format(accuracy_score(df_test.Sex, predicted)))
print("F1-measure: {0:6.2f}".format(f1_score(df_test.Sex, predicted, average='macro')))

Accuracy:   0.70
F1-measure:   0.67


Результаты:
* ngram_range=(4, 4) Accuracy:   0.73, F1-measure:   0.66
* ngram_range=(3, 3) Accuracy:   0.72, F1-measure:   0.68
* ngram_range=(2, 2) Accuracy:   0.67, F1-measure:   0.64
* ngram_range=(1, 1) Accuracy:   0.56, F1-measure:   0.55

#### Логистическая регрессия

In [22]:
Val1 = 2
Val2 = 4

clf1 = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(Val1,Val2))),
    ('clf1',  LogisticRegression())
    ])

In [23]:
clf1.fit(df_train.Names, df_train.Sex)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 4), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [24]:
predicted1 = clf1.predict(df_test.Names)

In [25]:
print("Accuracy: {0:6.2f}".format(accuracy_score(df_test.Sex, predicted1)))
print("F1-measure: {0:6.2f}".format(f1_score(df_test.Sex, predicted1, average='macro')))

Accuracy:   0.76
F1-measure:   0.74


Результаты:
* ngram_range=(4, 4) Accuracy:   0.64, F1-measure:   0.63
* ngram_range=(3, 3) Accuracy:   0.73, F1-measure:   0.71
* ngram_range=(2, 2) Accuracy:   0.75, F1-measure:   0.72
* ngram_range=(1, 1) Accuracy:   0.62, F1-measure:   0.60

Если использовать analyzer='char_wb' вместо analyzer='char', можно еще улучшить результат.

##  Часть 3. Нейронная сеть


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor